### **Setting GEE environnement**
Due to compatibility issues it is recommended to use a dedicated environment for the geemap library.

See instructions on how to do this here: https://geemap.org/installation/

In [3]:
# Install the libraries if they are not installed (you can remove these lines if you have already installed them)
# !pip install earthengine-api geemap

# Charging the libraries
import ee
import geemap

# Authenticate and initialize Earth Engine session
# ee.Authenticate()  # Only required the first time or when the access token expires
ee.Initialize()

### **Visualisation mode for one roi in one year_of_interest**

In [5]:
# Define the ROI and load the dataset
roi = ee.FeatureCollection('users/caslumali/UE801_Stage/MDD') # change the name of the ROIi : 'Cotriguacu' / 'Guaviare' / 'MDD' / 'Paragominas'
modisEVI = ee.ImageCollection('MODIS/061/MOD13Q1').select('EVI') # can be changed to 'EVI'

In [6]:
def calculate_annual_evi_sum(year, roi):
    start_date = ee.Date.fromYMD(year, 8, 1)  # Start of the climate year
    end_date = ee.Date.fromYMD(year + 1, 7, 31)  # End of the climate year
    filtered_year = modisEVI.filterDate(start_date, end_date).map(lambda image: image.clip(roi))
    annual_evi_sum = filtered_year.reduce(ee.Reducer.sum()).rename('EVI')
    return annual_evi_sum

def calculate_stats(image, roi):
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax().combine(reducer2=ee.Reducer.mean(), sharedInputs=True),
        geometry=roi.geometry(),
        scale=250, 
        maxPixels=1e9
    )
    return stats.getInfo()

# Year of interest for the EVI analysis
year_of_interest = 2000

# Calculate the annual cumulative EVI
annual_evi_sum = calculate_annual_evi_sum(year_of_interest, roi)

# Calculate and print EVI statistics
stats = calculate_stats(annual_evi_sum, roi)
print(f"EVI values for {year_of_interest} - {year_of_interest + 1}:\n", stats)

EVI values for 2000 - 2001:
 {'EVI_max': 150321, 'EVI_mean': 108908.08270316508, 'EVI_min': -3132}


In [7]:
# Calculate EVI statistics for the year of interest to generate dynamic visualization
stats = calculate_stats(annual_evi_sum, roi)

# Set display options using calculated minimum and maximum values
eviVis = {
    'min': stats['EVI_min'],
    'max': stats['EVI_max'],
    'palette': [
        'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718',
        '74a901', '66a000', '529400', '3e8601', '207401', '056201',
        '004c00', '023b01', '012e01', '011d01', '011301'
    ]
}

# View the EVI image on the map
Map = geemap.Map()
Map.addLayer(annual_evi_sum, eviVis, 'Annual EVI sum')
Map.centerObject(roi, 7) # Change the zoom depending on the area
Map

Map(center=[-11.980585283981338, -70.53595429489678], controls=(WidgetControl(options=['position', 'transparen…

### **Looping for export EVI for each ROI**

In [9]:
# Define ROIs in a list
# Export ROIs have a 10km buffer
rois = {
    'Cotriguacu': ee.FeatureCollection('users/caslumali/UE801_Stage/Cotriguacu_10km'),
    'Guaviare': ee.FeatureCollection('users/caslumali/UE801_Stage/Guaviare_10km'),
    'MDD': ee.FeatureCollection('users/caslumali/UE801_Stage/MDD_10km'),
    'Paragominas': ee.FeatureCollection('users/caslumali/UE801_Stage/Paragominas_10km'),
}

In [10]:
from datetime import datetime

# Get current year
current_year = datetime.now().year

# Loop for each ROI
for roi_name, roi in rois.items():
    print(f'Processing {roi_name}...')
    for year in range(2000, current_year):  # Goes to the current year
        # If we are in the current year and processing the last year, skip the combination
        if year == current_year - 1:
            continue  # Skip to the next iteration of the loop

        annualEVISum = calculate_annual_evi_sum(year, roi).toDouble()
        description = f'{roi_name}_EVI_Annual_Sum_{year}_{year + 1}'
        fileNamePrefix = f'{roi_name}_EVI_Annual_Sum_{year}_{year + 1}'
        
        task = ee.batch.Export.image.toDrive(
            image=annualEVISum,
            description=description,
            fileNamePrefix=fileNamePrefix,
            folder=f'{roi_name}_EVI_Annual',
            scale=250,
            region=roi.geometry(),
            crs='EPSG:4326',
            fileFormat='GeoTIFF',
            maxPixels=1e9
        )
        task.start()
        print(f'Export task started for {roi_name} in the year: {year}')


Processing Cotriguacu...
Export task started for Cotriguacu in the year: 2000
Export task started for Cotriguacu in the year: 2001
Export task started for Cotriguacu in the year: 2002
Export task started for Cotriguacu in the year: 2003
Export task started for Cotriguacu in the year: 2004
Export task started for Cotriguacu in the year: 2005
Export task started for Cotriguacu in the year: 2006
Export task started for Cotriguacu in the year: 2007
Export task started for Cotriguacu in the year: 2008
Export task started for Cotriguacu in the year: 2009
Export task started for Cotriguacu in the year: 2010
Export task started for Cotriguacu in the year: 2011
Export task started for Cotriguacu in the year: 2012
Export task started for Cotriguacu in the year: 2013
Export task started for Cotriguacu in the year: 2014
Export task started for Cotriguacu in the year: 2015
Export task started for Cotriguacu in the year: 2016
Export task started for Cotriguacu in the year: 2017
Export task started f